In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.image as mpimg
import random
# import seaborn as sns
import scipy.signal as ss
from scipy.special import i0
from scipy.ndimage import convolve1d
# from matplotlib import animation
# from matplotlib import rc as matrc
# matrc('animation', html='jshtml', embed_limit=20971520*2)
# import matplotlib.patches as patches

In [2]:
import Ipynb_importer
import familiarity_functions

importing Jupyter notebook from familiarity_functions.ipynb


In [3]:
# Generating the stimuli
img_dir = 'tang_stimuli/pattern_images/'

# in version 13；
# img_idx = np.linspace(0, 9500, num = 500, dtype = int, endpoint = False) + 2
# img_idx = np.arange(1,500, dtype=int)

# in version 14:
# the long bars are from 80 to 1599
num_in_this_phase = 500
img_idx_all = np.setdiff1d(np.arange(9500, dtype = int), np.arange(80, 1600, dtype = int))
step = int(np.ceil((9500 - 1600 + 80) / num_in_this_phase))
remainder = int(num_in_this_phase - len(img_idx_all) // step - 1)
img_idx = np.concatenate(([img_idx_all[i] for i in range(0, len(img_idx_all), step)], img_idx_all[1:remainder + 1]))

num_stimuli = len(img_idx)
img_wid = 160
img_len = 160
pooling_wid = 1
pooling_len = 1
img_s_wid = int(np.ceil(img_wid / pooling_wid))
img_s_len = int(np.ceil(img_len / pooling_len))
img = np.zeros((num_stimuli, img_s_wid, img_s_len))

wid_rf = 52
len_rf = 52
overlap = 18

def rgb2gray(rgb):
    gray = np.dot(rgb[...,:3], [0.299, 0.587, 0.144])
    gray = gray / np.max(gray)
    return gray

def center(img, size):
    indent =np.array((np.array(np.shape(img)) - size)/2).astype(int)
    return img[indent[0]:-(np.shape(img)[0]-size[0]-indent[0]), indent[1]:-(np.shape(img)[1]-size[1]-indent[1])]

# for i in range(num_stimuli):
    # img[i,:,:] = familiarity_functions.pooling(mpimg.imread(img_dir + '%d.png' %img_idx[i]), [pooling_wid, pooling_len])

num_row = int((img_s_wid - wid_rf)/overlap) + 1
num_col = int((img_s_len - len_rf)/overlap) + 1
num_hyper = num_row * num_col
print('Number of hypercolumns = %d' %num_hyper)
print('Number of stimuli = %d' %num_stimuli)
# print(img_idx)

Number of hypercolumns = 49
Number of stimuli = 500


In [4]:
# Parameters regarding the Gabor filters

gabor_size = [wid_rf, len_rf]

num_orient = 8
orientation = np.linspace(-np.pi/2, np.pi/2, num_orient, endpoint=False)

# in version 13:
# spat_freq = [0.5, 1.0] # similar to the derivative order
# sigma_deno = [10.0, 20.0] # similar to the aspect ratio

# in version 14:
spat_freq = [1.0]
sigma_deno = [10.0]

num_kernel = 2 * num_orient * len(spat_freq) * len(sigma_deno) # cos and sin

In [5]:
# Generate Gabor Filters
gabor_arr_cos = np.zeros((len(spat_freq) * len(sigma_deno), wid_rf, len_rf, num_orient))
gabor_arr_sin = np.zeros((len(spat_freq) * len(sigma_deno), wid_rf, len_rf, num_orient))
for i in range(len(spat_freq)):
    for j in range(len(sigma_deno)):
        gabor_arr_cos[i*len(sigma_deno)+j], gabor_arr_sin[i*len(sigma_deno)+j] = \
        familiarity_functions.gabor2D_Lee(gabor_size, orientation, sigma_deno[j], spat_freq[i])

In [6]:
# kernel order: cos, sin
gabor_arr = np.zeros((2 * len(spat_freq) * len(sigma_deno) * num_orient, wid_rf, len_rf))
for i in range(num_orient):
    gabor_arr[i * len(spat_freq) * len(sigma_deno):(i+1) * len(spat_freq) * len(sigma_deno)] = gabor_arr_cos[:,:,:,i]
    gabor_arr[(i+num_orient)*len(spat_freq)*len(sigma_deno):(i+1+num_orient) * len(spat_freq) * len(sigma_deno)] = gabor_arr_sin[:,:,:,i]

print(np.shape(gabor_arr))

(16, 52, 52)


In [7]:
# in version 14:
# 48 is the old number of kernels
r_in = np.load("test_on_pattern_22/r_in_all.npy")[img_idx]

In [19]:
def r(ui, beta, t=0.0):
    return np.maximum(ui - t, 0) * beta

def init_weights(num_in, num_out, mean_value):
    return mean_value * np.ones((num_in, num_out))

def Newton(f, y0, ts, save_ts, progress=True):
    ys = np.zeros((len(save_ts), len(y0)))
    y = y0
    dt = ts[1] - ts[0]
    k = -1
    skip = save_interval
    theta = y[-N_e:] # the initial theta
    
    count = np.zeros(N_e)
    count_img = np.zeros(num_stimuli)
    

    for i, t in enumerate(ts):
        if progress is True:
            print("Iteration: ", i)
        if i % skip == 0:
            k += 1
            ys[k] = y
        if i % save_output_interval == 0:
            np.save("phase_1_pattern_28/ys_%d"%int(i/tau_stim), ys)
            np.save("phase_1_pattern_28/count_%d"%int(i/tau_stim), count)
            np.save("phase_1_pattern_28/count_img_%d"%int(i/tau_stim), count_img)
            print("finish_stim: %d"%int(i/tau_stim))
        
        if i % int(tau_stim) == 0:
            # clear the synaptic input for a new stimulus
            y[:N_e + N_i] = 0
                       
        result_from_dydt = f(t, y, i, stim, theta)
        y[:-N_e] = y[:-N_e] + dt*result_from_dydt[:-N_e]
        y[-N_e:] = result_from_dydt[-N_e:]
        
        theta = y[-N_e:]
        
        # in version 16:
        y[:N_e + N_i] = np.maximum(y[:N_e + N_i], np.zeros(N_e + N_i))
        
        # Set excitatory synapses to non-negative
        wrp_flat = y[N_e + N_i:-N_e]
        assert len(wrp_flat) == N_e * N_e
        wrp_flat = np.maximum(wrp_flat, np.zeros(N_e * N_e))      
        wrp_flat = np.minimum(wrp_flat, wrp_max * np.ones(N_e * N_e))
        
#         y[N_e + N_i:-N_e] = wrp_flat
        
        # in version 27:
        wrp = np.reshape(wrp_flat, (N_e, N_e))
        wrp = (wrp.T/np.sum(wrp, axis=1) * weight_constant).T
        y[N_e + N_i:-N_e] = wrp.flatten()
        
                         

        
        if (i % int(tau_stim) == int(tau_stim)-1):
            # total pre-synaptic weight = constant
            wrp = np.reshape(y[N_e + N_i:-N_e], (N_e, N_e))
            wrp = (wrp.T/np.sum(wrp, axis=1) * weight_constant).T
            y[N_e + N_i:-N_e] = wrp.flatten()
            # count whether a neuron participates in this circuit
            re = r(y[:N_e], beta, threshold)
            idx = np.arange(N_e, dtype = int)[np.nonzero(np.int64(re > theta_BCM))]
            if len(idx) > 1:
                for j in range(len(idx)):
                    if len(np.intersect1d(familiarity_functions.choosing_inhibition_neighbors(idx[j], int(num_kernel/2), num_row, num_col, \
                                                                                              radius_inhibition), idx)) < len(idx):
                        count[idx[j]] += 1
                        count_img[i // int(tau_stim)] += 1
        
    ys[-1] = y
    
    return (save_ts, ys, count, count_img)

In [9]:
N_e = int(num_kernel/2) * num_row * num_col
print(N_e)

392


In [11]:
# All Parameters

# Parameters regarding time
tau_stim = 200.0 # time per stimulus
delta_t = 1.0 # simulation timestep = 1 ms
save_every_stim = 0.1
save_interval = int(tau_stim / save_every_stim) # save simulation state every save_interval steps.
# Resulting variables
T = tau_stim * num_stimuli # simulation time
n = int(round(T / delta_t))
N_repeats = int(round(tau_stim / delta_t))
ts = np.linspace(0.0, T, num=n, endpoint=False)
save_ts = np.linspace(0, T, num=int(n/save_interval)+1)

# time constant of synaptic input change
tau_ue = 0.2 # 100 times of this value
tau_ui = 0.1 # 100 times of this value

# number of inhibitory neurons
N_i = int(N_e / (len(spat_freq) * len(sigma_deno)))

# new variable
radius_spatial = 2
radius_inhibition = 1

N_neighbors = int(num_kernel/2) * (2*radius_spatial+1) * (2*radius_spatial+1)
print(N_neighbors)

# Parameters regarding the weight
# wrp_max = 25.0 / N_neighbors
# in version 26:
wrp_max = 50.0 / N_neighbors
wei = 1.0 / N_i
wie = 30.0 / ((2 * radius_inhibition + 1) * (2 * radius_inhibition + 1)*len(spat_freq) * len(sigma_deno))
wee = 5.0 / N_neighbors

tau_wrp = 200000.0 # 10 ** 4 times of this value
# decay = 0.1 ** (1/num_stimuli)
# in version 26:
decay = 0.05 ** (1/num_stimuli)

# scalars in the dynamical equation
gamma = 350.0



# threshold for firing
beta = 1.0
# threshold = np.mean(r_in) * wee * N_neighbors * gamma
# a very sensitive variable
threshold = 0.0

# noise in the dynamical equation
F = 0.0

# new variable
theta_cap = 40.0

# in version 14:
# theta_noise_sd = 10.0
# in version 15:
# in version 22:
theta_noise_ratio = 2.0
theta_BCM = np.load("test_on_pattern_22/threshold_BCM.npy")
# only in this version (13):
g1 = np.random.choice(N_e, size = N_e // 3, replace = False)
g2 = np.random.choice(np.setdiff1d(np.arange(N_e), g1), size = N_e // 3, replace = False)
g3 = np.setdiff1d(np.setdiff1d(np.arange(N_e), g1), g2)
theta_noise = np.empty(N_e)
theta_noise[g1] = theta_BCM[g1] / theta_noise_ratio
theta_noise[g2] = 0
theta_noise[g3] = -theta_BCM[g3] / theta_noise_ratio
# plt.plot(theta_noise)

# theta_noise = np.random.normal(loc=0.0, scale = theta_noise_sd, size=N_e)
theta_BCM = theta_BCM + theta_noise

weight_constant = wee * N_neighbors


print(threshold)

200
0.0


In [13]:
np.save("phase_1_pattern_28/theta_noise", theta_noise)

In [14]:
r_in_flat = r_in.flatten()
r_in_repeat = np.repeat(np.reshape(r_in_flat, (num_stimuli, N_e)), N_repeats, axis = 0)

In [15]:
# Initializing the weights
wrp0 = np.zeros((N_e, N_e))
for i in range(N_e):
    wrp0[i, familiarity_functions.choosing_neighbors(i, int(num_kernel/2), num_row, num_col, radius_spatial)] = wee

wei0 = wei * np.ones((N_e, N_i))
wie0 = wie * np.zeros((N_i, N_e))
# num_kernel * num_row * num_col
for i in range(N_i):
    kernel_num = int(i / (num_row * num_col))
    row_num = int((i - kernel_num * num_row * num_col) / num_col)
    col_num = int((i - kernel_num * num_row * num_col - row_num * num_col))
    for col in range(int(2 * radius_inhibition + 1)):
        if (col_num - radius_inhibition + col >= 0 and col_num - radius_inhibition + col <= num_col-1):
            for row in range(int(2 * radius_inhibition + 1)):
                if (row_num - radius_inhibition + row >= 0 and row_num - radius_inhibition + row <= num_row-1):
                    wie0[i,np.arange(kernel_num * len(spat_freq) * len(sigma_deno) * num_row * num_col+col_num - radius_inhibition + col\
                                + (row_num - radius_inhibition + row) * num_col, \
                                (kernel_num+1) * len(spat_freq) * len(sigma_deno) * num_row * num_col, num_row * num_col, dtype = int)] = wie

ue = np.zeros((N_e, 1))
ui = np.zeros((N_i, 1))
y0 = np.concatenate((ue.flatten(), ui.flatten(), wrp0.flatten(), theta_cap * np.ones(N_e) + theta_noise))

# assign variables only
duedt = np.zeros((N_e, 1))
duidt = np.zeros((N_i, 1))
Iffue = np.zeros((N_e, 1))
Iffue_mean = np.zeros((N_e, 1))
wrp = np.zeros((N_e, N_e))
dwrpdt = np.zeros((N_e, N_e))
rc = np.zeros((N_e))
stim = 0

In [16]:
save_output_num_stim = 10
save_output_interval = int(save_output_num_stim * tau_stim)

In [17]:
# in version 14
# theta_BCM_long = np.load("BCM_threshold_before_all_patterns.npy")
# theta_BCM = theta_BCM_long[2 * N_e:3 * N_e] + theta_noise

# in version 15:
# theta_BCM = np.load("test_on_pattern_15/threshold_BCM.npy")

In [20]:
def dydt(t, y, i, stim, theta):
    
    ue = y[:N_e]
    ui = y[N_e:N_e + N_i]
    re = r(ue, beta, threshold)
    ri = r(ui, beta)
    wrp = np.reshape(y[N_e + N_i:-N_e],(N_e, N_e))

    if (i % int(tau_stim) == 0):
        theta = i/tau_stim / (i/tau_stim+1) * theta + (gamma * r_in_repeat[i,:] + threshold) ** 2.0 / (i/tau_stim+1)
        theta = np.minimum(theta, theta_cap * np.ones(N_e)) + theta_noise
    
    Iffue_mean = gamma * r_in_repeat[i,:]
    Iffue = np.maximum(Iffue_mean + np.sqrt(F * Iffue_mean / delta_t) * np.random.normal(size=N_e), np.zeros(N_e))
    duedt = 1.0/tau_ue * (-ue + np.dot(wrp, re) - np.dot(wei0, ri) + Iffue) * 0.01
    duidt = 1.0/tau_ui * (-ui + np.dot(wie0, re)) * 0.01
    # BCM rule
    dwrpdt = np.zeros((N_e, N_e))
    for j in range(N_e):
        neighbors = familiarity_functions.choosing_neighbors(j, int(num_kernel/2), num_row, num_col, radius_spatial)
        dwrpdt[j, neighbors] = 1.0 / tau_wrp * re[j] * (re[j] - theta_BCM[j]) * re[neighbors] * (10 ** (-4)) * (decay ** (i // int(tau_stim)))
    
    # print(theta)
    return np.concatenate((duedt, duidt, dwrpdt.flatten(), theta))

t, ys, counts, counts_img = Newton(dydt, y0, ts, save_ts, progress=False)

Iteration:  0
finish_stim: 0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49
Iteration:  50
Iteration:  51
Iteration:  52
Iteration:  53
Iteration:  54
Iteration:  55
Iteration:  56
Iteration:  57
Iteration:  58
Iteration:  59
Iteration:  60
Iteration:  61
Iteration:  62
Iteration:  63
Iteration:  64
Iteration:  65
Itera

Iteration:  520
Iteration:  521
Iteration:  522
Iteration:  523
Iteration:  524
Iteration:  525
Iteration:  526
Iteration:  527
Iteration:  528
Iteration:  529
Iteration:  530
Iteration:  531
Iteration:  532
Iteration:  533
Iteration:  534
Iteration:  535
Iteration:  536
Iteration:  537
Iteration:  538
Iteration:  539
Iteration:  540
Iteration:  541
Iteration:  542
Iteration:  543
Iteration:  544
Iteration:  545
Iteration:  546
Iteration:  547
Iteration:  548
Iteration:  549
Iteration:  550
Iteration:  551
Iteration:  552
Iteration:  553
Iteration:  554
Iteration:  555
Iteration:  556
Iteration:  557
Iteration:  558
Iteration:  559
Iteration:  560
Iteration:  561
Iteration:  562
Iteration:  563
Iteration:  564
Iteration:  565
Iteration:  566
Iteration:  567
Iteration:  568
Iteration:  569
Iteration:  570
Iteration:  571
Iteration:  572
Iteration:  573
Iteration:  574
Iteration:  575
Iteration:  576
Iteration:  577
Iteration:  578
Iteration:  579
Iteration:  580
Iteration:  581
Iteratio

Iteration:  1034
Iteration:  1035
Iteration:  1036
Iteration:  1037
Iteration:  1038
Iteration:  1039
Iteration:  1040
Iteration:  1041
Iteration:  1042
Iteration:  1043
Iteration:  1044
Iteration:  1045
Iteration:  1046
Iteration:  1047
Iteration:  1048
Iteration:  1049
Iteration:  1050
Iteration:  1051
Iteration:  1052
Iteration:  1053
Iteration:  1054
Iteration:  1055
Iteration:  1056
Iteration:  1057
Iteration:  1058
Iteration:  1059
Iteration:  1060
Iteration:  1061
Iteration:  1062
Iteration:  1063
Iteration:  1064
Iteration:  1065
Iteration:  1066
Iteration:  1067
Iteration:  1068
Iteration:  1069
Iteration:  1070
Iteration:  1071
Iteration:  1072
Iteration:  1073
Iteration:  1074
Iteration:  1075
Iteration:  1076
Iteration:  1077
Iteration:  1078
Iteration:  1079
Iteration:  1080
Iteration:  1081
Iteration:  1082
Iteration:  1083
Iteration:  1084
Iteration:  1085
Iteration:  1086
Iteration:  1087
Iteration:  1088
Iteration:  1089
Iteration:  1090
Iteration:  1091
Iteration:  10

Iteration:  1520
Iteration:  1521
Iteration:  1522
Iteration:  1523
Iteration:  1524
Iteration:  1525
Iteration:  1526
Iteration:  1527
Iteration:  1528
Iteration:  1529
Iteration:  1530
Iteration:  1531
Iteration:  1532
Iteration:  1533
Iteration:  1534
Iteration:  1535
Iteration:  1536
Iteration:  1537
Iteration:  1538
Iteration:  1539
Iteration:  1540
Iteration:  1541
Iteration:  1542
Iteration:  1543
Iteration:  1544
Iteration:  1545
Iteration:  1546
Iteration:  1547
Iteration:  1548
Iteration:  1549
Iteration:  1550
Iteration:  1551
Iteration:  1552
Iteration:  1553
Iteration:  1554
Iteration:  1555
Iteration:  1556
Iteration:  1557
Iteration:  1558
Iteration:  1559
Iteration:  1560
Iteration:  1561
Iteration:  1562
Iteration:  1563
Iteration:  1564
Iteration:  1565
Iteration:  1566
Iteration:  1567
Iteration:  1568
Iteration:  1569
Iteration:  1570
Iteration:  1571
Iteration:  1572
Iteration:  1573
Iteration:  1574
Iteration:  1575
Iteration:  1576
Iteration:  1577
Iteration:  15

Iteration:  2002
Iteration:  2003
Iteration:  2004
Iteration:  2005
Iteration:  2006
Iteration:  2007
Iteration:  2008
Iteration:  2009
Iteration:  2010
Iteration:  2011
Iteration:  2012
Iteration:  2013
Iteration:  2014
Iteration:  2015
Iteration:  2016
Iteration:  2017
Iteration:  2018
Iteration:  2019
Iteration:  2020
Iteration:  2021
Iteration:  2022
Iteration:  2023
Iteration:  2024
Iteration:  2025
Iteration:  2026
Iteration:  2027
Iteration:  2028
Iteration:  2029
Iteration:  2030
Iteration:  2031
Iteration:  2032
Iteration:  2033
Iteration:  2034
Iteration:  2035
Iteration:  2036
Iteration:  2037
Iteration:  2038
Iteration:  2039
Iteration:  2040
Iteration:  2041
Iteration:  2042
Iteration:  2043
Iteration:  2044
Iteration:  2045
Iteration:  2046
Iteration:  2047
Iteration:  2048
Iteration:  2049
Iteration:  2050
Iteration:  2051
Iteration:  2052
Iteration:  2053
Iteration:  2054
Iteration:  2055
Iteration:  2056
Iteration:  2057
Iteration:  2058
Iteration:  2059
Iteration:  20

...

Iteration:  98038
Iteration:  98039
Iteration:  98040
Iteration:  98041
Iteration:  98042
Iteration:  98043
Iteration:  98044
Iteration:  98045
Iteration:  98046
Iteration:  98047
Iteration:  98048
Iteration:  98049
Iteration:  98050
Iteration:  98051
Iteration:  98052
Iteration:  98053
Iteration:  98054
Iteration:  98055
Iteration:  98056
Iteration:  98057
Iteration:  98058
Iteration:  98059
Iteration:  98060
Iteration:  98061
Iteration:  98062
Iteration:  98063
Iteration:  98064
Iteration:  98065
Iteration:  98066
Iteration:  98067
Iteration:  98068
Iteration:  98069
Iteration:  98070
Iteration:  98071
Iteration:  98072
Iteration:  98073
Iteration:  98074
Iteration:  98075
Iteration:  98076
Iteration:  98077
Iteration:  98078
Iteration:  98079
Iteration:  98080
Iteration:  98081
Iteration:  98082
Iteration:  98083
Iteration:  98084
Iteration:  98085
Iteration:  98086
Iteration:  98087
Iteration:  98088
Iteration:  98089
Iteration:  98090
Iteration:  98091
Iteration:  98092
Iteration:

Iteration:  98496
Iteration:  98497
Iteration:  98498
Iteration:  98499
Iteration:  98500
Iteration:  98501
Iteration:  98502
Iteration:  98503
Iteration:  98504
Iteration:  98505
Iteration:  98506
Iteration:  98507
Iteration:  98508
Iteration:  98509
Iteration:  98510
Iteration:  98511
Iteration:  98512
Iteration:  98513
Iteration:  98514
Iteration:  98515
Iteration:  98516
Iteration:  98517
Iteration:  98518
Iteration:  98519
Iteration:  98520
Iteration:  98521
Iteration:  98522
Iteration:  98523
Iteration:  98524
Iteration:  98525
Iteration:  98526
Iteration:  98527
Iteration:  98528
Iteration:  98529
Iteration:  98530
Iteration:  98531
Iteration:  98532
Iteration:  98533
Iteration:  98534
Iteration:  98535
Iteration:  98536
Iteration:  98537
Iteration:  98538
Iteration:  98539
Iteration:  98540
Iteration:  98541
Iteration:  98542
Iteration:  98543
Iteration:  98544
Iteration:  98545
Iteration:  98546
Iteration:  98547
Iteration:  98548
Iteration:  98549
Iteration:  98550
Iteration:

Iteration:  98953
Iteration:  98954
Iteration:  98955
Iteration:  98956
Iteration:  98957
Iteration:  98958
Iteration:  98959
Iteration:  98960
Iteration:  98961
Iteration:  98962
Iteration:  98963
Iteration:  98964
Iteration:  98965
Iteration:  98966
Iteration:  98967
Iteration:  98968
Iteration:  98969
Iteration:  98970
Iteration:  98971
Iteration:  98972
Iteration:  98973
Iteration:  98974
Iteration:  98975
Iteration:  98976
Iteration:  98977
Iteration:  98978
Iteration:  98979
Iteration:  98980
Iteration:  98981
Iteration:  98982
Iteration:  98983
Iteration:  98984
Iteration:  98985
Iteration:  98986
Iteration:  98987
Iteration:  98988
Iteration:  98989
Iteration:  98990
Iteration:  98991
Iteration:  98992
Iteration:  98993
Iteration:  98994
Iteration:  98995
Iteration:  98996
Iteration:  98997
Iteration:  98998
Iteration:  98999
Iteration:  99000
Iteration:  99001
Iteration:  99002
Iteration:  99003
Iteration:  99004
Iteration:  99005
Iteration:  99006
Iteration:  99007
Iteration:

Iteration:  99412
Iteration:  99413
Iteration:  99414
Iteration:  99415
Iteration:  99416
Iteration:  99417
Iteration:  99418
Iteration:  99419
Iteration:  99420
Iteration:  99421
Iteration:  99422
Iteration:  99423
Iteration:  99424
Iteration:  99425
Iteration:  99426
Iteration:  99427
Iteration:  99428
Iteration:  99429
Iteration:  99430
Iteration:  99431
Iteration:  99432
Iteration:  99433
Iteration:  99434
Iteration:  99435
Iteration:  99436
Iteration:  99437
Iteration:  99438
Iteration:  99439
Iteration:  99440
Iteration:  99441
Iteration:  99442
Iteration:  99443
Iteration:  99444
Iteration:  99445
Iteration:  99446
Iteration:  99447
Iteration:  99448
Iteration:  99449
Iteration:  99450
Iteration:  99451
Iteration:  99452
Iteration:  99453
Iteration:  99454
Iteration:  99455
Iteration:  99456
Iteration:  99457
Iteration:  99458
Iteration:  99459
Iteration:  99460
Iteration:  99461
Iteration:  99462
Iteration:  99463
Iteration:  99464
Iteration:  99465
Iteration:  99466
Iteration:

Iteration:  99871
Iteration:  99872
Iteration:  99873
Iteration:  99874
Iteration:  99875
Iteration:  99876
Iteration:  99877
Iteration:  99878
Iteration:  99879
Iteration:  99880
Iteration:  99881
Iteration:  99882
Iteration:  99883
Iteration:  99884
Iteration:  99885
Iteration:  99886
Iteration:  99887
Iteration:  99888
Iteration:  99889
Iteration:  99890
Iteration:  99891
Iteration:  99892
Iteration:  99893
Iteration:  99894
Iteration:  99895
Iteration:  99896
Iteration:  99897
Iteration:  99898
Iteration:  99899
Iteration:  99900
Iteration:  99901
Iteration:  99902
Iteration:  99903
Iteration:  99904
Iteration:  99905
Iteration:  99906
Iteration:  99907
Iteration:  99908
Iteration:  99909
Iteration:  99910
Iteration:  99911
Iteration:  99912
Iteration:  99913
Iteration:  99914
Iteration:  99915
Iteration:  99916
Iteration:  99917
Iteration:  99918
Iteration:  99919
Iteration:  99920
Iteration:  99921
Iteration:  99922
Iteration:  99923
Iteration:  99924
Iteration:  99925
Iteration:

In [21]:
np.save('phase_1_pattern_28/ys_500', ys)
np.save('phase_1_pattern_28/counts_500', counts)
np.save('phase_1_pattern_28/counts_img_500', counts_img)